<a href="https://colab.research.google.com/github/seongwoojang1123/TMJ-osteoarthritis-diagnosis-/blob/main/4_Image_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os
import shutil
import pandas as pd
import random
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Image Masking

In [ ]:
# 원본 및 출력 디렉토리 설정
source_dir = r'C:/TMJ OA/source_data'
output_base_dir = r'C:/TMJ OA/masked_data'

# 텍스트 영역 마스킹 함수
def remove_text_only(image):
    masked_image = image.copy()
    masked_image[0:20, :] = 0  # 상단 10~30 픽셀 범위 마스킹
    height = masked_image.shape[0]
    masked_image[height - 20:height - 0, :] = 0  # 하단 30~10 픽셀 범위 마스킹
    masked_image[:, 0:95] = 0  # 좌측 0~90 픽셀 범위 마스킹
    width = masked_image.shape[1]
    masked_image[:, width - 95:] = 0  # 우측 90 픽셀 범위 마스킹
    return masked_image

# 이미지 전처리 및 저장 함수
def process_images_in_directory(input_dir, output_base_dir):
    for root, dirs, files in os.walk(input_dir):
        relative_path = os.path.relpath(root, input_dir)
        output_dir = os.path.join(output_base_dir, relative_path)

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        print(f"Processing directory: {output_dir}")

        for filename in files:
            # 확장자 조건 제거 (모든 파일을 시도)
            image_path = os.path.join(root, filename)
            image = cv2.imread(image_path)

            if image is not None:
                # 텍스트 영역 제거
                masked_image = remove_text_only(image)
                output_path = os.path.join(output_dir, filename)
                success = cv2.imwrite(output_path, masked_image)

                if success:
                    print(f"Image successfully saved to {output_path}")
                else:
                    print(f"Failed to save image at: {output_path}")
            else:
                print(f"Failed to load image {image_path} - It might not be a valid image file.")

# 전체 이미지 전처리 수행
process_images_in_directory(source_dir, output_base_dir)

Image Cropping

In [ ]:
# 이미지 로드
image_path = 'C:/TMJ OA/masked_data/20703917 1.JPG'
img = Image.open(image_path)

# 크롭 영역 정의 (좌표: (left, upper, right, lower))
crop_box = (110, 45, 265, 250)

# 크롭 수행
cropped_img = img.crop(crop_box)

# 크롭된 이미지 시각화
plt.imshow(cropped_img)
plt.title("Cropped Image")
plt.axis("off")
plt.show()

Cropping Image Save

In [ ]:
# 폴더 경로 설정
input_folder = 'C:/TMJ OA/masked_data'  # 원본 이미지 폴더
output_folder = 'C:/TMJ OA/masked_2_data' # 결과 저장 폴더

# 크롭 영역 정의 (모든 이미지에 동일한 영역 적용)
crop_box = (110, 45, 265, 250)

# 출력 폴더 생성
os.makedirs(output_folder, exist_ok=True)

# 이미지 일괄 처리
for filename in os.listdir(input_folder):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):  # 이미지 파일 필터링
        # 이미지 로드
        image_path = os.path.join(input_folder, filename)
        img = Image.open(image_path)

        # 크롭 수행
        cropped_img = img.crop(crop_box)

        # 크롭된 이미지 저장
        output_path = os.path.join(output_folder, filename)
        cropped_img.save(output_path)

print(f"All images cropped and saved to {output_folder}")

Image Classification

In [ ]:
# Excel 파일 경로 및 이미지 경로 설정
excel_path = 'C:/Users/mook6/Downloads/20240819 CBCT MRI 데이터 정리_수정1.xlsx'
source_dir = 'C:/TMJ OA/masked_data'
train_dir = 'C:/TMJ OA/train_data'
validation_dir = 'C:/TMJ OA/validation_data'
test_dir = 'C:/TMJ OA/test_data'

# Excel 파일 로드 및 OA 상태를 딕셔너리로 저장 (ID별로 분류)
oa_data = pd.read_excel(excel_path)
oa_dict = {row['Patient ID']: (row['CBCT_Rt OA (Osteoarthritis) (0=normal, 1=OA)'],
                               row['CBCT_Lt OA  (Osteoarthritis) (0=normal, 1=OA)'])
           for _, row in oa_data.iterrows()}

# 이미지 필터링 (우측: 1.JPG, 좌측: 3.JPG 이미지만 선택)
all_files = [f for f in os.listdir(source_dir) if f.endswith('1.JPG') or f.endswith('3.JPG')]

# 레이블에 따른 분류
rt_oa_files = [f for f in all_files if f.endswith('1.JPG') and oa_dict.get(int(f.split(' ')[0]), (0, 0))[0] == 1]
rt_normal_files = [f for f in all_files if f.endswith('1.JPG') and oa_dict.get(int(f.split(' ')[0]), (0, 0))[0] == 0]
lt_oa_files = [f for f in all_files if f.endswith('3.JPG') and oa_dict.get(int(f.split(' ')[0]), (0, 0))[1] == 1]
lt_normal_files = [f for f in all_files if f.endswith('3.JPG') and oa_dict.get(int(f.split(' ')[0]), (0, 0))[1] == 0]


# 함수: 각 레이블 비율에 맞춰 데이터를 분할
def split_data(files, train_ratio=0.6, val_ratio=0.2):
    train_files, temp_files = train_test_split(files, train_size=train_ratio, random_state=42)
    val_files, test_files = train_test_split(temp_files, test_size=val_ratio / (1 - train_ratio), random_state=42)
    return train_files, val_files, test_files


# 각 레이블에 따라 분할
train_rt_oa, val_rt_oa, test_rt_oa = split_data(rt_oa_files)
train_rt_normal, val_rt_normal, test_rt_normal = split_data(rt_normal_files)
train_lt_oa, val_lt_oa, test_lt_oa = split_data(lt_oa_files)
train_lt_normal, val_lt_normal, test_lt_normal = split_data(lt_normal_files)

# 분할된 파일들을 합치기
train_files = train_rt_oa + train_rt_normal + train_lt_oa + train_lt_normal
validation_files = val_rt_oa + val_rt_normal + val_lt_oa + val_lt_normal
test_files = test_rt_oa + test_rt_normal + test_lt_oa + test_lt_normal

# 학습, 검증, 테스트 폴더 생성
for label_type in ['Rt_Normal', 'Lt_Normal', 'Rt_OA', 'Lt_OA']:
    os.makedirs(os.path.join(train_dir, label_type), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, label_type), exist_ok=True)
    os.makedirs(os.path.join(test_dir, label_type), exist_ok=True)


# 파일 복사 함수
def copy_files(file_list, target_dir, oa_dict):
    for filename in file_list:
        file_path = os.path.join(source_dir, filename)
        img_id = int(filename.split(' ')[0])  # ID가 첫 부분에 있다고 가정

        # 우측/좌측 및 OA 상태에 따른 폴더 설정
        if filename.endswith('1.JPG'):  # 우측
            oa_status = oa_dict.get(img_id, (0, 0))[0]  # 우측 OA 상태
            label = 'Rt_OA' if oa_status == 1 else 'Rt_Normal'
        elif filename.endswith('3.JPG'):  # 좌측
            oa_status = oa_dict.get(img_id, (0, 0))[1]  # 좌측 OA 상태
            label = 'Lt_OA' if oa_status == 1 else 'Lt_Normal'

        # 파일 복사
        target_label_dir = os.path.join(target_dir, label)
        shutil.copy(file_path, os.path.join(target_label_dir, filename))


# 파일 복사 수행
copy_files(train_files, train_dir, oa_dict)
copy_files(validation_files, validation_dir, oa_dict)
copy_files(test_files, test_dir, oa_dict)

# 결과 출력
print(f"학습 데이터로 복사된 파일 수: {len(train_files)}")
print(f"검증 데이터로 복사된 파일 수: {len(validation_files)}")
print(f"테스트 데이터로 복사된 파일 수: {len(test_files)}")

Image Checking

In [ ]:
# Excel 파일 경로 및 이미지 폴더 경로 설정
excel_path = 'C:/Users/mook6/Downloads/20240819 CBCT MRI 데이터 정리_수정1.xlsx'
base_dir = 'C:/TMJ OA/'
train_dir = os.path.join(base_dir, 'train_data')
validation_dir = os.path.join(base_dir, 'validation_data')
test_dir = os.path.join(base_dir, 'test_data')

# Excel 파일 로드 및 불필요한 텍스트와 NaN 제거
oa_data = pd.read_excel(excel_path, usecols=['Patient ID', 'NAME', 'CBCT_Rt OA (Osteoarthritis) (0=normal, 1=OA)', 'CBCT_Lt OA  (Osteoarthritis) (0=normal, 1=OA)'])

# Patient ID와 Name 모두에 대해 NaN 값이 없는 유효한 데이터만 필터링
oa_data = oa_data.dropna(subset=['Patient ID', 'NAME'])  # NaN 값 제거
oa_data = oa_data[pd.to_numeric(oa_data['Patient ID'], errors='coerce').notna()]  # Patient ID가 숫자인 행만 남김
oa_data['Patient ID'] = oa_data['Patient ID'].astype(str)  # 문자열로 변환하여 일관성 유지

# Patient ID와 Name 정보 딕셔너리 생성
oa_info = {row['Patient ID']: row for _, row in oa_data.iterrows()}
excel_patient_ids = set(oa_info.keys())

# 이미지 폴더에서 모든 Patient ID 추출
all_image_patient_ids = set()
for folder in [train_dir, validation_dir, test_dir]:
    if os.path.exists(folder):
        for label_type in os.listdir(folder):
            label_path = os.path.join(folder, label_type)
            if os.path.isdir(label_path):
                all_image_patient_ids.update(file.split(' ')[0] for file in os.listdir(label_path))

# Excel에 있지만 이미지 폴더에 없는 Patient ID와 NAME 정보
missing_in_images = excel_patient_ids - all_image_patient_ids
missing_in_images_info = {pid: oa_info[pid]['NAME'] for pid in missing_in_images}

# 이미지 폴더에 있지만 Excel에 없는 Patient ID
missing_in_excel = all_image_patient_ids - excel_patient_ids
missing_in_excel_count = len(missing_in_excel)

# 결과 출력
print(f"Excel에는 있지만 이미지 폴더에 없는 Patient ID 개수: {len(missing_in_images)}")
print("Excel에는 있지만 이미지 폴더에 없는 Patient ID와 NAME 정보:", missing_in_images_info)

print(f"\n이미지 폴더에는 있지만 Excel에 없는 Patient ID 개수: {missing_in_excel_count}")
print("이미지 폴더에는 있지만 Excel에 없는 Patient ID:", missing_in_excel)